### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

### This demo uses the latest pillow package to show the rectangular bounding box around the face, so please upgrade the pillow package using the command below:

In [ ]:
!pip install Pillow==8.4

## Importing Useful Python Libraries or Packages 

In [ ]:
import io
import datetime
import pandas as pd
from PIL import Image
import requests
import io
import glob, os, sys, time, uuid

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw

from video_indexer import VideoIndexer
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import TrainingStatusType
from msrest.authentication import CognitiveServicesCredentials

### Documentation
- https://docs.microsoft.com/en-us/azure/azure-video-analyzer/video-analyzer-for-media-docs/video-indexer-use-apis

### Set up the Video Analyzer Portal Login:
- https://www.videoindexer.ai/account
- https://www.videoindexer.ai/media/library  - All uploaded video

### Get API Subscription:
https://api-portal.videoindexer.ai/

### Article
- https://medium.com/microsoftazure/visual-brand-detection-with-azure-video-indexer-cd65330b908c

In [ ]:
CONFIG = {
    'SUBSCRIPTION_KEY': 'YOUR VIDEO INDEXER SUBSCRIPTION KEY',
    'LOCATION': 'trial',
    'ACCOUNT_ID': 'YOUR VIDEO INDEXER ACCOUNT ID'
}

video_analysis = VideoIndexer(
    vi_subscription_key=CONFIG['SUBSCRIPTION_KEY'],
    vi_location=CONFIG['LOCATION'],
    vi_account_id=CONFIG['ACCOUNT_ID']
)

### Location Parameter
 - For paid service, please use service region i.e. westus2, eastus, etc.
 - For trial or free service, just use "trial" as I have used above. 

In [ ]:
video_analysis.check_access_token()

## Note: Please upload your video to Azure Video Analyzer/Indexer service and replace the code below with your own video ID

### To get video ID, check on the video in the Library and you will get the video ID, appended to the URL. An example is shown here:
* Video URL:
https://www.videoindexer.ai/accounts/d1629197-588b-40a7-98fa-e19785ca082e/videos/2240904ed2
* Video ID: 2240904ed2
* If you don't have a video at this point, you can download this video and upload it at the Video Indexer portal:
https://github.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/blob/main/resources/video-indexer-demo-video.mp4

In [ ]:
video_id = 'ENTER YOUR VIDEO ID'

In [ ]:
video_analysis.get_video_info(video_id)

In [ ]:
info = video_analysis.get_video_info(video_id, video_language='English')

## Processing RAW Json 
### Getting a list of thumbnails where we find human face

In [ ]:
if len(info['videos'][0]['insights']['faces'][0]['thumbnails']):
    print("We found {} faces in this video.".format(str(len(info['videos'][0]['insights']['faces'][0]['thumbnails']))))

In [ ]:
info['videos'][0]['insights']['faces'][0]['thumbnails']

## Getting Thumbnail ID from the Analysis JSON 

In [ ]:
images = []
img_raw = []
img_strs = []
for each_thumb in info['videos'][0]['insights']['faces'][0]['thumbnails']:
    if 'fileName' in each_thumb and 'id' in each_thumb:
        file_name = each_thumb['fileName']
        thumb_id = each_thumb['id']
        img_code = video_analysis.get_thumbnail_from_video_indexer(video_id,  thumb_id)
        img_strs.append(img_code)
        img_stream = io.BytesIO(img_code)
        img_raw.append(img_stream)
        img = Image.open(img_stream)
        images.append(img)

## Now, let's view the face-specific thumbnails 

In [ ]:
for img in images:
    print(img.info)
    plt.figure()
    plt.imshow(img)

## Do you remember these 4 faces from the previous demo?

## Let's extract and save these face thumbnails to the local disk 
- Download from Cloud

In [ ]:
i = 1
for img in images:
    print(type(img))
    img.save('human-face' + str(i) + '.jpg')
    i= i+ 1

## Verify the download process 

In [ ]:
!ls human-face*.jpg

## Getting thumbnail from the SDK 

In [ ]:
# Enter one of the thumbnail output you got from the previous cell, 
# under the "Getting Thumbnail ID from the Analysis JSON" section.
thumbnail_id='ENTER THUMBNAIL HERE'

In [ ]:
img_code = video_analysis.get_thumbnail_from_video_indexer(video_id,  thumbnail_id)
print(img_code)

## Converting encoded image to visible image

In [ ]:
img_code = video_analysis.get_thumbnail_from_video_indexer(video_id,  thumbnail_id)
img_stream = io.BytesIO(img_code)
img = Image.open(img_stream)
imshow(img)

## Getting thumbnails using Keyframes 

In [ ]:
keyframes = []
for shot in info["videos"][0]["insights"]["shots"]:
    for keyframe in shot["keyFrames"]:
        keyframes.append(keyframe["instances"][0]['thumbnailId'])

In [ ]:
for keyframe in keyframes:
    img_str = video_analysis.get_thumbnail_from_video_indexer(video_id,  keyframe)

## Emotion from the Video Analyzer

In [ ]:
info['summarizedInsights']['sentiments']

In [ ]:
info['summarizedInsights']['emotions']

# Collecting Faces from  Video Analyzer

In [ ]:
AVKASH_FACE_KEY = "YOUR FACE SERVICE KEY"
AVKASH_FACE_ENDPOINT = "YOUR FACE SERVICE ENDPOINT"

In [ ]:
# Create a client
face_client = FaceClient(AVKASH_FACE_ENDPOINT, CognitiveServicesCredentials(AVKASH_FACE_KEY))

In [ ]:
face_client.api_version

https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/Face/DetectIdentifyFace.py

# Creating Person Model Based on Faces in the Video

Note: this section is the same as the Facial recognition Demo. You can jump to the next section "Additional Resource: uploading a video from local disk to Video Analyzer portal" if you don't want to perform personal model again.

### We have already downloaded and saved the face thumbnails in the previous steps
- We will be using those face thumbnails here to build the Person model

In [ ]:
PERSON_GROUP_ID = str(uuid.uuid4())
person_group_name = 'person-avkash'

# Note if this UUID already used earlier, you will get an error 

In [ ]:
## This code is taken from Azure Face SDK 
## ---------------------------------------
def build_person_group(client, person_group_id, pgp_name):
    print('Create and build a person group...')
    # Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
    print('Person group ID:', person_group_id)
    client.person_group.create(person_group_id = person_group_id, name=person_group_id)

    # Create a person group person.
    human_person = client.person_group_person.create(person_group_id, pgp_name)
    # Find all jpeg human images in working directory.
    human_face_images = [file for file in glob.glob('*.jpg') if file.startswith("human-face")]
    # Add images to a Person object
    for image_p in human_face_images:
        with open(image_p, 'rb') as w:
            client.person_group_person.add_face_from_stream(person_group_id, human_person.person_id, w)

    # Train the person group, after a Person object with many images were added to it.
    client.person_group.train(person_group_id)

    # Wait for training to finish.
    while (True):
        training_status = client.person_group.get_training_status(person_group_id)
        print("Training status: {}.".format(training_status.status))
        if (training_status.status is TrainingStatusType.succeeded):
            break
        elif (training_status.status is TrainingStatusType.failed):
            client.person_group.delete(person_group_id=PERSON_GROUP_ID)
            sys.exit('Training the person group has failed.')
        time.sleep(5)


In [ ]:
build_person_group(face_client, PERSON_GROUP_ID, person_group_name)

# Making sure the Person model has faces and they all belong to the same person


In [ ]:
'''
Detect all faces in query image list, then add their face IDs to a new list.
'''
def detect_faces(client, query_images_list):
    print('Detecting faces in query images list...')

    face_ids = {} # Keep track of the image ID and the related image in a dictionary
    for image_name in query_images_list:
        image = open(image_name, 'rb') # BufferedReader
        print("Opening image: ", image.name)
        time.sleep(5)

        # Detect the faces in the query images list one at a time, returns list[DetectedFace]
        faces = client.face.detect_with_stream(image)  

        # Add all detected face IDs to a list
        for face in faces:
            print('Face ID', face.face_id, 'found in image', os.path.splitext(image.name)[0]+'.jpg')
            # Add the ID to a dictionary with image name as a key.
            # This assumes there is only one face per image (since you can't have duplicate keys)
            face_ids[image.name] = face.face_id

    return face_ids

In [ ]:
test_images = [file for file in glob.glob('*.jpg') if file.startswith("human-face")]

In [ ]:
test_images

In [ ]:
ids = detect_faces(face_client, test_images)

In [ ]:
ids

### Verifying that 2 random images from the list belong to the same person
- #### Note: So far we have not used the face recognition part, only face detection.

In [ ]:
# Verification example for faces of the same person.
verify_result = face_client.face.verify_face_to_face(ids['human-face1.jpg'], ids['human-face2.jpg'])

In [ ]:
if verify_result.is_identical:
    print("Faces are of the same (Positive) person, similarity confidence: {}.".format(verify_result.confidence))
else:
    print("Faces are of different (Negative) persons, similarity confidence: {}.".format(verify_result.confidence))

## Matching face from ID card with face from Video Analyzer 

In [ ]:
def show_image_in_cell(face_url):
    response = requests.get(face_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(10,5))
    plt.imshow(img)
    plt.show()

In [ ]:
dl_source_url = 'https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/ca-dl-sample.png'


In [ ]:
show_image_in_cell(dl_source_url)

In [ ]:
## -------
## Reading file locally
## -------
# If I had image file locally, I would have used the following method
# dl_image = open('/Users/avkashchauhan99/work/avkash/udacity/cal-dl.png', 'rb')
# dl_faces = face_client.face.detect_with_stream(dl_image)  

In [ ]:
dl_faces = face_client.face.detect_with_url(dl_source_url) 

## Viewing Face ID and then saving it into the list of already saved Face IDs

In [ ]:
for face in dl_faces:
    print('Face ID', face.face_id, 'found in image', dl_source_url)
    # Add the ID to a dictionary with image name as a key.
    # This assumes there is only one face per image (since you can't have duplicate keys)
    ids['ca-dl-sample.png'] = face.face_id

## Now, we have 4 + 1 = 5 Face IDs in our Face ID list

In [ ]:
ids

## Perform face verification between the Face ID from the identity card and one of the Face IDs from the video

In [ ]:
# Verification example for faces of the same person.
dl_verify_result = face_client.face.verify_face_to_face(ids['human-face4.jpg'], ids['ca-dl-sample.png'])

In [ ]:
if dl_verify_result.is_identical:
    print("Faces are of the same (Positive) person, similarity confidence: {}.".format(dl_verify_result.confidence))
else:
    print("Faces are of different (Negative) persons, similarity confidence: {}.".format(dl_verify_result.confidence))

In [ ]:
ids['ca-dl-sample.png']

In [ ]:
ids.values()

In [ ]:
dl_faces[0].face_rectangle.as_dict()

In [ ]:
# TAKEN FROM THE Azure SDK Sample
# Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))

In [ ]:
def drawFaceRectangles(source_file, detected_face_object) :
    # Download the image from the url
    response = requests.get(source_file)
    img = Image.open(BytesIO(response.content))
    # Draw a red box around every detected faces
    draw = ImageDraw.Draw(img)
    for face in detected_face_object:
        draw.rectangle(getRectangle(face), outline='red', width = 10)
    return img

In [ ]:
drawFaceRectangles(dl_source_url, dl_faces)

## Matching Face ID from the identity card with Video Analyzer Person Model 

In [ ]:
# A list of Face ID
ids

## Using the face ID from the identify card and matching the identity with the Person Group model

In [ ]:
# Enter the face ID of ca-dl-sample.png from the output of the cell above
get_the_face_id_from_the_driving_license = 'ENTER FACE ID HERE'

In [ ]:
person_gp_results = face_client.face.identify([get_the_face_id_from_the_driving_license], PERSON_GROUP_ID)

In [ ]:
for result in person_gp_results:
    for candidate in result.candidates:
        print("The Identity match confidence is {}".format(candidate.confidence))

# Additional Resource:
## Uploading a video from local disk to Video Analyzer portal

- My sample video is saved in this workspace environment, and the "input_filename" variable in the cell below is currently set to its path in this environment.
- If you want to upload your own video from this workspace environment to Video Analyzer portal, first upload your video to the workspace, then change the "input_filename" variable to reflect the name of your video.

In [ ]:
uploaded_video_id = video_analysis.upload_to_video_indexer(
   input_filename='/home/workspace/video-indexer-demo-video.mp4',
   video_name='avkash-11-second',  # unique identifier for video in Video Indexer platform
   video_language='English'
)

In [ ]:
uploaded_video_id

In [ ]:
info = video_analysis.get_video_info(uploaded_video_id, video_language='English')

In [ ]:
info

### You should be able to see this video at the Video Analyzer portal.
# End of demo